In [2]:
import altair as alt
from pathlib import Path
import json
import pandas as pd
import numpy as np

alt.renderers.enable('altair_viewer', inline=True)

RendererRegistry.enable('altair_viewer')

# Process Data

In [515]:
results = {
    'no-static': ('../results/17_09_51-08_2020/data.csv', '../results/17_09_51-08_2020/sim.csv'),
    'static': ('../results/14_15_10-08_2020/data.csv', '../results/14_15_10-08_2020/sim2.csv'),
    'unrolled': ('../results/14_17_26-08_2020/data.csv', '../results/14_17_26-08_2020/sim.csv')
}
data_lib = {}

for k, v in results.items():
    futil_hls_data = pd.read_csv(v[0])
    futil_latency = pd.read_csv(v[1])
    futil_hls_data = futil_hls_data[futil_hls_data['type'] != 'hls_total']
    for _, row in futil_latency.iterrows():
        bench_filt = futil_hls_data['benchmark'] == row[0]
        type_filt = futil_hls_data['type'] == 'futil'
        futil_hls_data.loc[(bench_filt) & (type_filt), 'latency'] = row[1]
    futil_hls_data = futil_hls_data.sort_values(by='benchmark').reset_index(drop=True)
    futil_hls_data['benchmark'] = futil_hls_data['benchmark'].apply(lambda x: x.split('-')[-1])
    data_lib[k] = futil_hls_data

# merge futil latency data with the main dataframe
# futil_hls_data = pd.read_csv('../results/14_15_10-08_2020/data.csv')
# futil_latency = pd.read_csv('../results/14_15_10-08_2020/sim.csv')
# futil_hls_data = futil_hls_data[futil_hls_data['type'] != 'hls_total']
# for _, row in futil_latency.iterrows():
#     bench_filt = futil_hls_data['benchmark'] == row[0]
#     type_filt = futil_hls_data['type'] == 'futil'
#     futil_hls_data.loc[(bench_filt) & (type_filt), 'latency'] = row[1]
# futil_hls_data = futil_hls_data.sort_values(by='benchmark').reset_index(drop=True)
# futil_hls_data


# Timed Futil vs VHLS Graph

In [522]:
df = data_lib['static']
seq = pd.read_csv('../small_seq.csv')

data = pd.DataFrame()
for bench in df['benchmark'].unique():
    rows = df[df['benchmark'] == bench]
    hls = rows[rows['type'] == 'hls']
    futil = rows[rows['type'] == 'futil']
    data = data.append({
        'benchmark': bench,
        'type': f"lut",
        'value': futil['lut'].values[0] / hls['lut'].values[0],
        'meet_timing': futil['meet_timing'].values[0],
        'equal': 1
    }, ignore_index=True)
#     data = data.append({
#         'benchmark': bench,
#         'type': f"dsp",
#         'value': futil['dsp'].values[0] / hls['dsp'].values[0],
#         'meet_timing': futil['meet_timing'].values[0],
#         'equal': 1
#     }, ignore_index=True)
    data = data.append({
        'benchmark': bench,
        'type': "latency",
        'value': futil['latency'].values[0] / hls['latency'].values[0],
        'meet_timing': futil['meet_timing'].values[0],
        'equal': 1
    }, ignore_index=True)
#     data = data.append({
#         'benchmark': b,
#         'type': "lseq",
#         'value': seq.loc[seq['benchmark'] == bench,'latency'].values[0] / hls['latency'].values[0],
#         'meet_timing': futil['meet_timing'].values[0],
#         'equal': 1
#     }, ignore_index=True)

data['lut_mean'] = data.loc[data['type'] == 'lut', 'value'].mean()
data['latency_mean'] = data.loc[data['type'] == 'latency', 'value'].mean()

bars = alt.Chart().mark_bar().encode(
    x=alt.X('type', axis=alt.Axis(labels=False, title=None, ticks=False)),
    y=alt.Y('value',
            scale=alt.Scale(type='log'),
            axis=alt.Axis(title="VHLS / Calyx")),
    tooltip=['value', 'latency_mean', 'lut_mean'],
    color=alt.condition('datum.meet_timing == 0', alt.ColorValue('red'), 'type'),
)

vline = alt.Chart().mark_rule(color='black', size=0).encode(y='equal')
# meanLutLine = alt.Chart().mark_rule(color='blue').encode(y='lut_mean')
# meanLatLine = alt.Chart().mark_rule(color='green').encode(y='latency_mean')
height=300
alt.layer(bars, vline, data=data).facet(
    column=alt.Column('benchmark', header=alt.Header(labelOrient='bottom')),
    spacing=1,
).configure_title(
    orient='bottom'
).configure_view(
    continuousHeight=height
).configure_bar(
    y2=height/2
)

data

,benchmark,equal,meet_timing,type,value,lut_mean,latency_mean
0,2mm,1.0,1.0,lut,4.454739,1.785513,3.415057
1,2mm,1.0,1.0,latency,4.434104,1.785513,3.415057
2,3mm,1.0,1.0,lut,0.785714,1.785513,3.415057
3,3mm,1.0,1.0,latency,3.472734,1.785513,3.415057
4,atax,1.0,1.0,lut,1.885246,1.785513,3.415057
5,atax,1.0,1.0,latency,3.022378,1.785513,3.415057
6,bicg,1.0,1.0,lut,0.972477,1.785513,3.415057
7,bicg,1.0,1.0,latency,5.518349,1.785513,3.415057
8,cholesky,1.0,1.0,lut,0.151980,1.785513,3.415057
9,cholesky,1.0,1.0,latency,1.835379,1.785513,3.415057


# Futil
Static vs No-static

In [517]:
static = data_lib['static']
no_static = data_lib['no-static']
# df = pd.concat([static, no_static], keys=['static', 'no-static']).reset_index().drop(['level_1'], axis='columns').rename({'level_0': 'timing'}, axis='columns')
df = static[static['type'] == 'futil']
base = df.drop(columns=['source', 'meet_timing', 'dsp', 'type']).sort_values(by='benchmark').reset_index(drop=True)
other = no_static[no_static['type'] == 'futil'].sort_values(by='benchmark').reset_index(drop=True)
base['lut'] /= other['lut']
base['latency'] /= other['latency']
# base['dsp'] /= other['dsp']
data = base.melt(['benchmark'])
height=300
bars = alt.Chart().mark_bar().encode(
    x=alt.X('variable', axis=alt.Axis(labels=False, title=None, ticks=False)),
    y=alt.Y('value',
            scale=alt.Scale(type='log', base=2, domain=(2 ** -0.5, 2 ** 0.5)),
            axis=alt.Axis(title="Latency Sensitive / Latency Insensitive")),
    color=alt.Color('variable', scale=alt.Scale(scheme='tableau10'))
)

alt.layer(bars, data=data).facet(
    column=alt.Column(
        'benchmark',
        header=alt.Header(labelOrient='bottom', labelAngle=0, labelAnchor='middle', labelPadding=20),
    ),
    spacing=1
).configure_view(
    continuousHeight=height,
    strokeWidth=1,
    fill='white',
).configure_bar(
    y2=height/2
)


alt.FacetChart(...)

In [518]:
systolic_data = pd.read_csv('../systolic-experiments/scan/data.csv')
systolic_lat = pd.read_csv('../systolic-experiments/scan/systolic.csv')
hls = pd.read_csv('../results/17_21_55-08_2020/data.csv')
hls = hls[hls['type'] == 'hls'].sort_values(by='benchmark').reset_index(drop=True)
systolic_data = systolic_data[systolic_data['type'] == 'futil'].sort_values(by='benchmark').drop(columns=['meet_timing', 'source', 'type']).reset_index(drop=True)
systolic_data = systolic_data.drop([0, 4, 6]).reset_index(drop=True)
systolic_data['latency'] = systolic_lat['latency']
systolic_data['latency_hls'] = hls['latency']
# systolic_data['hls_dsp'] = hls['dsp']
# systolic_data['hls_lut'] = hls['lut']
systolic_data['lut / 10'] = systolic_data['lut'] / 10
systolic_data['benchmark'] = systolic_data['benchmark'].apply(lambda x: "{} x {}".format(x.split('_')[1], x.split('_')[1]))
# systolic_data['lat_expect'] = [26 * (x + (np.sqrt(x))) for x in range(1,9)]
d = systolic_data.melt(['benchmark'])
alt.Chart(d).mark_bar().encode(
    x=alt.X('variable', axis=alt.Axis(labels=False, ticks=False,title=None)),
    y='value',
    color='variable',
    column=alt.Column(
        'benchmark', 
        header=alt.Header(labelOrient='bottom', labelAngle=0, labelAnchor='middle', labelPadding=20),
        spacing=1
    ),
    tooltip=['value']
).transform_filter(
    'datum.variable != "lut"'
).configure_view(
    continuousHeight=200
)

alt.Chart(...)

In [178]:
systolic_data['dsp_diff'] / list(range(1, 9))

0     2.0
1     8.0
2    15.0
3    24.0
4    35.0
5    48.0
6    63.0
7    80.0
Name: dsp_diff, dtype: float64

In [526]:
data_lib['unrolled'].drop(columns=['meet_timing', 'source']).melt(['benchmark'], var_name='type', value_name='value')

,benchmark,type,value
0,2mm,type,hls
1,2mm,type,futil
2,atax,type,hls
3,atax,type,futil
4,gemm,type,hls
...,...,...,...
59,mvt,latency,969
60,syr2k,latency,NaN
61,syr2k,latency,10641
62,syrk,latency,NaN
